## Sestamibi Scan Concordance Analysis

This section analyzes the **performance of Sestamibi scans** in localizing parathyroid pathology among patients with localizing scans (Left or Right).

### **Summary Metrics**

- **Exact Match:** Imaging identified the exact surgical gland.  
- **Side Match:** Imaging identified the correct side but not the exact gland.  
- **Partial Match:** Imaging identified at least one of multiple surgical glands but not all, or had a side match in multi-gland disease without gland match.  
- **Incorrect Localization:** Imaging findings did not match surgical findings.

The table below summarizes the **counts and percentages** for each match category among all localized Sestamibi scans in the dataset.

In [ ]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [2]:
# Define modality and relevant columns
modality = 'Sestamibi'
concordance_col = 'Sestamibi Calculated Concordance'
imaging_side_col = 'Sestamibi Imaging Side'

In [3]:
# Filter: only scans that localized to Left or Right
localized_sestamibi = preop_imaging_df[preop_imaging_df[imaging_side_col].isin(['Left', 'Right'])].copy()
total_localized_sestamibi = localized_sestamibi.shape[0]

In [7]:
# Calculate counts from pre-calculated concordance
exact_count = localized_sestamibi[localized_sestamibi[concordance_col] == 'Yes (Exact)'].shape[0]
side_count = localized_sestamibi[localized_sestamibi[concordance_col] == 'Yes (Side)'].shape[0]
partial_count = localized_sestamibi[localized_sestamibi[concordance_col] == 'Yes (Partial)'].shape[0]
incorrect_count = localized_sestamibi[localized_sestamibi[concordance_col] == 'Incorrect Localization'].shape[0]

# Calculate percentages
exact_pct = round((exact_count / total_localized_sestamibi) * 100, 2) if total_localized_sestamibi > 0 else None 
side_pct = round((side_count / total_localized_sestamibi) * 100, 2) if total_localized_sestamibi > 0 else None
partial_pct = round((partial_count / total_localized_sestamibi) * 100, 2) if total_localized_sestamibi > 0 else None
incorrect_pct = round((incorrect_count / total_localized_sestamibi) * 100, 2) if total_localized_sestamibi > 0 else None 

# Combine exact + side for total side match
total_side_match_count = exact_count + side_count
total_side_match_pct = round((total_side_match_count / total_localized_sestamibi) * 100, 2) if total_localized_sestamibi > 0 else None



In [10]:
# Build clean summary dataframe
side_concordance_sestamibi_df = pd.DataFrame({
    'Match Type': [
        'Exact Match',
        'Side Match',
        'Partial Match',
        'Incorrect Localization'
    ],
    'Count': [
        exact_count,
        side_count,
        partial_count,
        incorrect_count
    ],
    'Percentage': [
        f"{exact_pct:.2f}%" if exact_pct is not None else 'N/A',
        f"{side_pct:.2f}%" if side_pct is not None else 'N/A',
        f"{partial_pct:.2f}%" if partial_pct is not None else 'N/A',
        f"{incorrect_pct:.2f}%" if incorrect_pct is not None else 'N/A'
    ]
})

print(f"Total localized Sestamibi scans: {total_localized_sestamibi}")
display(side_concordance_sestamibi_df)

Total localized Sestamibi scans: 113


,Match Type,Count,Percentage
0,Exact Match,46,40.71%
1,Side Match,58,51.33%
2,Partial Match,8,7.08%
3,Incorrect Localization,1,0.88%
